In [1]:
from langgraph.graph import StateGraph , START , END 
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from typing import Annotated , TypedDict 
import operator
from pydantic import BaseModel

In [2]:
os.environ["OPENROUTER_API_KEY"] = "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"

In [3]:
llm = ChatOpenAI(
    temperature=0.2,
    openai_api_base = "https://openrouter.ai/api/v1",
    openai_api_key = os.getenv("OPENROUTER_API_KEY"),
    model = "openai/gpt-4o-mini",
)

In [4]:
class PostState(TypedDict):
    prompt:str
    image_urls:Annotated[list[str],operator.add]
    post_content:str
    post_title:str
    post_hashtags:list[str]

In [47]:
import json
with open("./variables.json",'r') as file:
    content = file.read()
variables = json.loads(content)

In [48]:
class polish_output(BaseModel):
    original_prompt:str
    polished_prompt:str

def polish_prompt(state:PostState)->PostState:
    prompt = PromptTemplate.from_template(
        template = """
            The user is using an automated agent to post on {platform} about their platform herth . {about_herth} . 
            The user wants to create a post and they have provided a very casual prompt , your job is to tailor that prompt for {platform} to ensure that the post creation agent does a good job. 

            User's prompt:
            {prompt} 
        """
    )
    llm_structured = llm.with_structured_output(polish_output)
    prompt_value = prompt.invoke({"about_herth":variables["about_herth"],"platform":variables["platform"],"prompt":state["prompt"]})
    response = llm_structured.invoke(prompt_value)
    print(response)
    state['prompt'] = response.polished_prompt 
    return state

# response = polish_prompt({"prompt":"Post about our new posting feature in about 1 paragraph max"})

In [49]:
from typing import Annotated
from pydantic import BaseModel
from langchain_core.prompts import PromptTemplate

class generated_content_structure(BaseModel):
    content: str
    title: Annotated[str, "Generate a catchy, engaging title for the post"]
    hashtags: list[str]

def generate_content(state: dict) -> dict:
    prompt = PromptTemplate.from_template(
        """
        You are an automated agent that writes viral social media posts for {platform} about the platform Herth.

        About Herth:
        {about_herth}

        The user wants to create a post with maximum engagement.
        Write:
        1. The full post content.
        2. A catchy title for the post.
        3. 3–5 relevant hashtags.

        User’s prompt:
        {prompt}
        """
    )

    llm_structured = llm.with_structured_output(generated_content_structure)

    prompt_value = prompt.invoke({
        "about_herth": variables["about_herth"],
        "platform": variables["platform"],
        "prompt": state["prompt"]
    })

    response = llm_structured.invoke(prompt_value)

    print(response)
    state["post_content"] = response.content
    state["post_title"] = response.title
    state["post_hashtags"] = response.hashtags
    return state


# generate_content({"prompt":response['prompt']})

In [50]:
class check_generation:
    ai_generated:int
    authentic:int
    professional:int
    
def calculate_ai_generated(state:PostState)->PostState:
    return state
def calculate_authenticity(state:PostState)->PostState:
    return state
def calculate_professionalism(state:PostState)->PostState:
    return state

In [51]:
# making a state graph
checkpointer = MemorySaver()

graph = StateGraph(PostState)

graph.add_node("polish_prompt",polish_prompt)
graph.add_node("generate_content",generate_content)


# graph.add_node("calculate_ai_generated",calculate_ai_generated)
# graph.add_node("calculate_authenticity",calculate_authenticity)
# graph.add_node("calculate_professionalism",calculate_professionalism)

graph.add_edge(START,"polish_prompt")
graph.add_edge("polish_prompt","generate_content")
graph.add_edge("generate_content",END)

workflow = graph.compile(checkpointer = checkpointer)


In [53]:
config = {"configurable":{"thread_id":1}}
final_state = workflow.invoke({"prompt":"I am introducing a new feature to sell food "},config = config)

original_prompt='I am introducing a new feature to sell food' polished_prompt="🌟 Exciting News from Herth! 🌟 \n\nHello, Herth community! We're thrilled to announce a brand new feature that empowers our women entrepreneurs to sell food directly through our platform! 🍽️✨ \n\nThis addition is designed to help you showcase your culinary creations, connect with food lovers, and expand your business in a vibrant community that champions women-led ventures. \n\nWhether you're a seasoned chef or just starting out, this feature will provide you with the tools and visibility you need to thrive. Join us in this delicious journey and let's make entrepreneurship more accessible and community-driven! \n\nWe can't wait to see what you create! Share your thoughts and let us know how this feature can support your food business. 💬❤️ \n\n#Herth #WomenEntrepreneurs #FoodBusiness #CommunityDriven"
content="🌟 Exciting News from Herth! 🌟 \n\nHello, Herth community! We're thrilled to announce a brand new feat

In [54]:
import praw
load_dotenv() 
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    username=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD"),
    user_agent=os.getenv("USER_AGENT")
)

print("Logged in as:", reddit.user.me())

# ---- POST TO SUBREDDIT ----
subreddit = reddit.subreddit("test")  # change to your target subreddit
# title = "Testing Reddit API Post 🧠"
# body = "This post was made automatically via Python using PRAW."

title = final_state['post_title']
content = final_state['post_content']

subreddit.submit(title=title, selftext=content)
print("Post submitted!")

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.